In [ ]:
import numpy as np
import elboflow as ef
import tensorflow as tf
import scipy.stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Generate some data for linear regression
np.random.seed(1)
num_samples = 100
num_dims = 2

feature_means = np.random.normal(0, 1, num_dims)
x = ef.add_bias(np.random.normal(0, 1, (num_samples, num_dims)) + feature_means)
theta = np.random.normal(0, 1, num_dims + 1)
predictor = np.dot(x, theta)
tau = np.random.gamma(1)
y = predictor + np.random.normal(0, 1 / np.sqrt(tau), num_samples)

In [ ]:
class LinearModel(ef.Model):
    def __init__(self, x, y, create_optimizer=None, create_session=None):
        super(LinearModel, self).__init__(x, y, create_optimizer=create_optimizer, 
                                          create_session=create_session)
    
    def setup(self, x, y):
        # Define the factors
        q_theta = ef.MultiNormalDistribution(
            ef.get_variable('theta_mean', num_dims + 1),
            ef.get_positive_definite_variable('theta_log_precision', (num_dims + 1, num_dims + 1))
        )
        q_tau = ef.GammaDistribution(
            ef.get_positive_variable('tau_shape', []),
            ef.get_positive_variable('tau_scale', []),
        )
        
        # Compute the likelihood and joint distribution
        log_likelihood = tf.reduce_sum(ef.NormalDistribution.linear_log_likelihood(x, y, q_theta, q_tau))
            
        return log_likelihood, {
            'theta': q_theta,
            'tau': q_tau
        }, {
            'theta': ef.MultiNormalDistribution(np.zeros(num_dims + 1), 1e-3 * np.eye(num_dims + 1)).log_proba,
            'tau': ef.GammaDistribution(1e-3, 1e-3).log_proba
        }
    
model = LinearModel(x, y)
model.run(model.elbo)

In [ ]:
trace = model.optimize(2000, [model.loss], tqdm=tqdm_notebook)
plt.plot(trace[model.loss])
plt.yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

model.plot_comparison('theta', theta, ax=ax1)
model.plot_proba('tau', reference=tau, ax=ax2)

In [ ]:
model.plot_cov('theta')